In [49]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [50]:
# Device configuration: set the device to GPU if available, otherwise use CPU
# Konfigurasi perangkat: atur perangkat ke GPU jika tersedia, jika tidak gunakan CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [51]:
# Hyper parameters: defining the main parameters for training the model
# Hyperparameter: mendefinisikan parameter utama untuk melatih model
num_epochs = 100  # Number of epochs for training; CIFAR-10 often requires more epochs to converge
# Jumlah epoch untuk pelatihan; CIFAR-10 sering membutuhkan lebih banyak epoch untuk konvergensi
num_classes = 10  # Total number of classes in the CIFAR-10 dataset (0-9)
batch_size = 256  # Increased batch size for better training efficiency
# Ukuran batch yang ditingkatkan untuk efisiensi pelatihan yang lebih baik
learning_rate = 0.001  # Learning rate for the optimizer
# Laju pembelajaran untuk optimizer

In [52]:
# Data augmentation and normalization for training data
# Augmentasi data dan normalisasi untuk data pelatihan
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),  # Randomly crop the image with padding to increase variety
    # Memotong gambar secara acak dengan padding untuk meningkatkan variasi
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    # Membalik gambar secara acak secara horizontal
    transforms.ToTensor(),  # Convert image to PyTorch tensor
    # Mengonversi gambar ke tensor PyTorch
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))  # Normalize with CIFAR-10 mean and std
    # Normalisasi dengan rata-rata dan deviasi standar CIFAR-10
])

In [53]:
# Data transformation for test set (no augmentation, only normalization)
# Transformasi data untuk set pengujian (tanpa augmentasi, hanya normalisasi)
transform_test = transforms.Compose([
    transforms.ToTensor(),  # Convert image to tensor
    # Mengonversi gambar ke tensor
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))  # Normalize using the same mean and std
    # Normalisasi menggunakan rata-rata dan deviasi standar yang sama
])

In [54]:
# Loading CIFAR-10 dataset
# Memuat dataset CIFAR-10
train_dataset = torchvision.datasets.CIFAR10(root='./data',
                                             train=True,  # Load training data
                                             transform=transform_train,  # Apply training transformations
                                             download=True)  # Download the dataset if not available locally

test_dataset = torchvision.datasets.CIFAR10(root='./data',
                                            train=False,  # Load testing data
                                            transform=transform_test)  # Apply testing transformations

# Data loader: Load datasets in batches for training and testing
# Pemuat data: Memuat dataset dalam batch untuk pelatihan dan pengujian
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,  # Number of samples per batch
                                           shuffle=True)  # Shuffle the training data for randomness

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,  # Number of samples per batch
                                          shuffle=False)  # No need to shuffle test data

Files already downloaded and verified


In [55]:
# Deep Convolutional Neural Network (CNN) for CIFAR-10 classification
# Jaringan Saraf Konvolusional (CNN) dalam untuk klasifikasi CIFAR-10
class CIFAR10Net(nn.Module):
    def __init__(self, num_classes=10):
        super(CIFAR10Net, self).__init__()
        # Define the first convolutional block
        # Mendefinisikan blok konvolusi pertama
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),  # First convolution layer with 64 filters
            # Lapisan konvolusi pertama dengan 64 filter
            nn.BatchNorm2d(64),  # Batch normalization for the first layer
            # Normalisasi batch untuk lapisan pertama
            nn.ReLU(),  # Activation function to introduce non-linearity
            # Fungsi aktivasi untuk memperkenalkan non-linearitas
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),  # Second convolution layer with 64 filters
            # Lapisan konvolusi kedua dengan 64 filter
            nn.BatchNorm2d(64),  # Batch normalization for the second layer
            # Normalisasi batch untuk lapisan kedua
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # Max pooling to reduce spatial dimensions by half
            # Max pooling untuk mengurangi dimensi spasial hingga setengah
        )
        # Define the second convolutional block
        # Mendefinisikan blok konvolusi kedua
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),  # First convolution layer with 128 filters
            # Lapisan konvolusi pertama dengan 128 filter
            nn.BatchNorm2d(128),  # Batch normalization for the first layer
            # Normalisasi batch untuk lapisan pertama
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),  # Second convolution layer with 128 filters
            # Lapisan konvolusi kedua dengan 128 filter
            nn.BatchNorm2d(128),  # Batch normalization for the second layer
            # Normalisasi batch untuk lapisan kedua
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # Max pooling to reduce spatial dimensions by half
            # Max pooling untuk mengurangi dimensi spasial hingga setengah
        )
        # Define the third convolutional block
        # Mendefinisikan blok konvolusi ketiga
        self.layer3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),  # First convolution layer with 256 filters
            # Lapisan konvolusi pertama dengan 256 filter
            nn.BatchNorm2d(256),  # Batch normalization for the first layer
            # Normalisasi batch untuk lapisan pertama
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),  # Second convolution layer with 256 filters
            # Lapisan konvolusi kedua dengan 256 filter
            nn.BatchNorm2d(256),  # Batch normalization for the second layer
            # Normalisasi batch untuk lapisan kedua
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # Max pooling to reduce spatial dimensions by half
            # Max pooling untuk mengurangi dimensi spasial hingga setengah
        )
        # Define the fourth convolutional block
        # Mendefinisikan blok konvolusi keempat
        self.layer4 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),  # First convolution layer with 512 filters
            # Lapisan konvolusi pertama dengan 512 filter
            nn.BatchNorm2d(512),  # Batch normalization for the first layer
            # Normalisasi batch untuk lapisan pertama
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),  # Second convolution layer with 512 filters
            # Lapisan konvolusi kedua dengan 512 filter
            nn.BatchNorm2d(512),  # Batch normalization for the second layer
            # Normalisasi batch untuk lapisan kedua
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # Max pooling to reduce spatial dimensions by half
            # Max pooling untuk mengurangi dimensi spasial hingga setengah
        )
        # Dropout layer to prevent overfitting
        # Lapisan dropout untuk mencegah overfitting
        self.dropout = nn.Dropout(0.5)  # Dropout with 50% probability
        # Dropout dengan probabilitas 50%
        # Fully connected layers for classification
        # Lapisan sepenuhnya terhubung untuk klasifikasi
        self.fc1 = nn.Linear(512 * 2 * 2, 512)  # Input is flattened from 512*2*2 features
        # Masukan diratakan dari 512*2*2 fitur
        self.fc2 = nn.Linear(512, num_classes)  # Final output layer producing num_classes outputs
        # Lapisan keluaran akhir menghasilkan keluaran jumlah kelas

    def forward(self, x):
        out = self.layer1(x)  # Forward pass through layer 1
        # Proses maju melalui lapisan 1
        out = self.layer2(out)  # Forward pass through layer 2
        # Proses maju melalui lapisan 2
        out = self.layer3(out)  # Forward pass through layer 3
        # Proses maju melalui lapisan 3
        out = self.layer4(out)  # Forward pass through layer 4
        # Proses maju melalui lapisan 4
        out = out.view(out.size(0), -1)  # Flatten the tensor for fully connected layers
        # Ratakan tensor untuk lapisan sepenuhnya terhubung
        out = self.dropout(out)  # Apply dropout to prevent overfitting
        # Terapkan dropout untuk mencegah overfitting
        out = self.fc1(out)  # First fully connected layer
        # Lapisan sepenuhnya terhubung pertama
        out = self.fc2(out)  # Final output layer
        # Lapisan keluaran akhir
        return out  # Return the output predictions
        # Mengembalikan prediksi keluaran

In [56]:
# Initialize the model and move it to the configured device
# Inisialisasi model dan pindahkan ke perangkat yang dikonfigurasi
model = CIFAR10Net(num_classes).to(device)

# Loss function and optimizer definition
# Definisi fungsi kerugian dan optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for multi-class classification
# Kerugian cross-entropy untuk klasifikasi multi-kelas
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)  # AdamW optimizer
# Optimizer AdamW

# Learning rate scheduler for adjusting learning rate during training
# Penjadwal laju pembelajaran untuk menyesuaikan laju pembelajaran selama pelatihan
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)  # Cosine annealing for smoother convergence
# Pengurangan cosinus untuk konvergensi yang lebih halus

In [57]:
# Training the model
# Melatih model
total_step = len(train_loader)  # Total number of steps (batches) per epoch
# Jumlah total langkah (batch) per epoch
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    
    cumulative_loss = 0.0  # Initialize cumulative loss for the epoch
    
    # Atur model ke mode pelatihan
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)  # Move images to the configured device (GPU or CPU)
        # Pindahkan gambar ke perangkat yang dikonfigurasi (GPU atau CPU)
        labels = labels.to(device)  # Move labels to the configured device
        # Pindahkan label ke perangkat yang dikonfigurasi
        
        # Forward pass: compute model predictions
        # Proses maju: hitung prediksi model
        outputs = model(images)  # Get model predictions for the input images
        # Dapatkan prediksi model untuk gambar masukan
        loss = criterion(outputs, labels)  # Compute the loss using the predicted and actual labels
        # Hitung kerugian menggunakan label yang diprediksi dan yang sebenarnya
        
        # Backward pass: compute gradients and update weights
        # Proses mundur: hitung gradien dan perbarui bobot
        optimizer.zero_grad()  # Clear old gradients from the optimizer
        # Hapus gradien lama dari optimizer
        loss.backward()  # Backpropagation: compute gradients based on the loss
        # Backpropagation: hitung gradien berdasarkan kerugian
        optimizer.step()  # Update model parameters using computed gradients
        # Perbarui parameter model menggunakan gradien yang dihitung
        
        cumulative_loss += loss.item()  # Accumulate the loss
        
    # Calculate average loss for the epoch
    avg_loss = cumulative_loss / total_step

    # Log the epoch number and average loss
    print(f'Epoch [{epoch + 1}/{num_epochs}], Avg Loss: {avg_loss:.4f}')

    # Step the scheduler at the end of each epoch to adjust learning rate
    # Langkah penjadwal di akhir setiap epoch untuk menyesuaikan laju pembelajaran
    scheduler.step()

Epoch [1/100], Avg Loss: 2.1724
Epoch [2/100], Avg Loss: 1.4898
Epoch [3/100], Avg Loss: 1.0900
Epoch [4/100], Avg Loss: 0.8539
Epoch [5/100], Avg Loss: 0.7054
Epoch [6/100], Avg Loss: 0.6210
Epoch [7/100], Avg Loss: 0.5496
Epoch [8/100], Avg Loss: 0.5000
Epoch [9/100], Avg Loss: 0.4539
Epoch [10/100], Avg Loss: 0.4154
Epoch [11/100], Avg Loss: 0.3820
Epoch [12/100], Avg Loss: 0.3506
Epoch [13/100], Avg Loss: 0.3395
Epoch [14/100], Avg Loss: 0.3041
Epoch [15/100], Avg Loss: 0.2834
Epoch [16/100], Avg Loss: 0.2712
Epoch [17/100], Avg Loss: 0.2470
Epoch [18/100], Avg Loss: 0.2283
Epoch [19/100], Avg Loss: 0.2109
Epoch [20/100], Avg Loss: 0.1945
Epoch [21/100], Avg Loss: 0.1820
Epoch [22/100], Avg Loss: 0.1676
Epoch [23/100], Avg Loss: 0.1565
Epoch [24/100], Avg Loss: 0.1405
Epoch [25/100], Avg Loss: 0.1302
Epoch [26/100], Avg Loss: 0.1211
Epoch [27/100], Avg Loss: 0.1109
Epoch [28/100], Avg Loss: 0.1009
Epoch [29/100], Avg Loss: 0.0892
Epoch [30/100], Avg Loss: 0.0840
Epoch [31/100], Avg

In [58]:
# Test the model on the test dataset
# Uji model pada dataset pengujian
model.eval()  # Set model to evaluation mode
# Atur model ke mode evaluasi
with torch.no_grad():  # No need to compute gradients during evaluation
    # Tidak perlu menghitung gradien selama evaluasi
    correct = 0  # Initialize correct predictions count
    # Inisialisasi hitungan prediksi yang benar
    total = 0  # Initialize total images count
    # Inisialisasi total jumlah gambar
    for images, labels in test_loader:
        images = images.to(device)  # Move images to the configured device
        # Pindahkan gambar ke perangkat yang dikonfigurasi
        labels = labels.to(device)  # Move labels to the configured device
        # Pindahkan label ke perangkat yang dikonfigurasi
        outputs = model(images)  # Get model predictions
        # Dapatkan prediksi model
        _, predicted = torch.max(outputs.data, 1)  # Get the class with the highest predicted score
        # Dapatkan kelas dengan skor prediksi tertinggi
        total += labels.size(0)  # Increment total number of images
        # Tambahkan total jumlah gambar
        correct += (predicted == labels).sum().item()  # Count the number of correct predictions
        # Hitung jumlah prediksi yang benar

    # Print test accuracy
    # Cetak akurasi pengujian
    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 90.21 %


In [59]:
# Save the model checkpoint after training
# Simpan checkpoint model setelah pelatihan
torch.save(model.state_dict(), 'cifar10_model.ckpt')  # Save the model's state (weights) to a file
# Simpan status model (bobot) ke file
